### Solves the 1D Wave equation with MDM and position Verlet algorithm

In [1]:
import numpy as np
from core import *

In [2]:
import matplotlib.pyplot as plt
from matplotlib import animation, rcParams
from IPython.display import HTML

In [3]:
rcParams['animation.html'] = 'html5'

In [4]:
def animate_wave(solution):
    fig = plt.figure(figsize=(8,6))
    ax = plt.gca()
    ax.set_xlim((0, 1))
    ax.set_ylim((-1.5, 1.5))
    ax.set_xlabel('x')
    ax.set_ylabel('u(x)')
    plt.grid(True)
    line, = ax.plot([], [], lw=2)
    plt.close()

    def animate(i):
        line.set_data(xgrid, solution[:, i])
        ax.set_title('1D Wave Equation\nt = {:01.2f}'.format(i * dt))
        return line,
   
    return animation.FuncAnimation(fig, animate, frames=solution.shape[1], interval=50, blit=True)

In [5]:
# Spatial discretization

# Order of accuracy (spatial)
k = 2
# Number of cells
m = 50

# Left boundary
a = 0
# Right boundary
b = 1

# Step length
dx = (b - a) / m

verlet = False  # If verlet = 0 then it uses 4th order accurate for time (FR)

# 1D Staggered grid
xgrid = np.append(np.insert(np.arange(a+dx/2, b, dx), 0, a), b)

# Mimetic operator (Laplacian)
L = lap1D(k, m, dx)

# Wave propagation speed
c = 2.  # (T/p) Tension over density

# "Force" function
F = lambda x : (c**2) * L * x  # c^2 DivGrad x

# Simulation time
TIME = 1.

# Temporal discretization based on CFL condition
dt = dx / (2 * c)  # dt = h on Young's paper

# Initial condition
ICU = lambda x : np.sin(np.pi * x)  # Initial position of particles
ICV = lambda x : np.zeros(m+1)  # Initial velocity of particles

uold = ICU(xgrid)
vold = ICV(xgrid)
vold = np.append(vold, vold[-1])

theta = 1. / (2. - 2.**(1./3.))  # From Peter Young's paper

Nt = int(TIME/dt)
solution = np.zeros((uold.shape[0], Nt))

# Time integration loop
for t in range(0, Nt):
    # Apply "position Verlet" algorithm (2nd-order in time)
    if verlet:
        uold = uold + 0.5 * dt * vold
        vnew = vold + dt * F(uold)
        unew = uold + 0.5 * dt * vnew
    # Apply "Forest-Ruth" algorithm (4th-order in time)
    else:
        unew = uold + theta * 0.5 * dt * vold
        vnew = vold + theta * dt * F(unew)
        unew = unew + (1 - theta) * 0.5 * dt * vnew
        vnew = vnew + (1 - 2*theta) * dt * F(unew)
        unew = unew + (1 - theta) * 0.5 * dt * vnew
        vnew = vnew + theta * dt * F(unew)
        unew = unew + theta * 0.5 * dt * vnew
    
    uold = unew
    vold = vnew
    
    solution[:, t] = unew

/Users/nutrik/Documents/Science/scodes/Python/pymole/core/div1D.py:53: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  D[i, j:j+k] = coeffs
/Users/nutrik/Documents/Science/scodes/Python/pymole/core/grad1D.py:52: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  G[i, j:j+k] = coeffs
/Users/nutrik/Documents/Science/scodes/Python/pymole/core/grad1D.py:72: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  A[i, 0:q] = coeffs
/Users/nutrik/Documents/Science/scodes/Python/pymole/core/grad1D.py:78: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  G[0:p, 0:q] = A
/Users/nutrik/Documents/Science/scodes/Python/pymole/core/grad1D.py:95: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. l

In [6]:
anim = animate_wave(solution)

In [7]:
anim